In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer 
%matplotlib inline
from sklearn.feature_extraction import text as TEXT
import re
from nltk.tokenize import RegexpTokenizer,wordpunct_tokenize, sent_tokenize
import os
from gensim.models import ldamulticore
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models, similarities, matutils
from sklearn.decomposition import TruncatedSVD
from nltk.tokenize import RegexpTokenizer
from nltk.stem import LancasterStemmer, PorterStemmer
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
import nltk
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Work cells

In [32]:
df_xls=pd.read_excel('C:/Users/52100722/Desktop/chatlog_honeywell/chat_log.xlsx',usecols=['Message','SessionID','Origin'],sheet_name='Chat_csv')
df_xls.Message.dropna(inplace=True)

In [33]:
df_xls.head(20)


,SessionID,Origin,Message
0,144209,SYSTEM,{H Eugene Talbott} [JOINED SESSION]
1,144209,SYSTEM,[Virgilio Dela Pena] ASSIGNED SESSION
2,144209,SYSTEM,NEW SESSION CREATED
3,144209,H Eugene Talbott ...,H&I
4,144209,SYSTEM,[Virgilio Dela Pena] ACCEPTED SESSION
5,144209,SYSTEM,[Virgilio Dela Pena] JOINED SESSION
6,144209,SYSTEM,NaN
7,144209,Virgilio Dela Pena,Thank you for chatting with us today. How can ...
8,144209,H Eugene Talbott ...,Good Morning Virgilio - I have a case open on ...
9,144209,SYSTEM,NaN


In [ ]:
if df_xls.SessionID

In [4]:
document=[]
for i in range(len(df_xls.Message)):
    document.append(str(df_xls.Message.iloc[i]))
    

In [5]:
lemmer=WordNetLemmatizer()
stemmer = PorterStemmer()
regex=RegexpTokenizer("\\b[a-zA-Z][a-zA-Z][a-zA-Z]+\\b")

def stem_tokens(tokens, stemmer):
    stemmed = []
    
    for item in tokens:
        #stemmed.append(stemmer.stem(item))
        stemmed.append(lemmer.lemmatize(item,"v"))
    return stemmed

def tokenize(text):
    text=text.lower()
    text=re.sub("[\(\[].*?[\)\]]", "", text)
    
    tokens = regex.tokenize(text)
   
    stems = stem_tokens(tokens, stemmer)
    return stems

stop=['virgilio', 'dela', 'pena','thank', 'how', 'good', 'morning','marilyn',
      'lista', 'selim', 'atakturk', 'samantha', 'wagner','this', 'can', 'okay',
      'miss', 'spallo', 'would', 'so', 'cigna', 'yes', 'the','eugene', 'talbott', 'join', 'session', 'virgilio',
      'assign', 'new', 'create', 'accept', 'chat','jhoana','cruz',"umali","andrew","klaudine","bettina","ocampo","carla"
      ,"alejandrino","cresencia","bustos","allan",'website','contact','records''online','record','waiting','web',
             'new','click','site','since','thanks','happy','david',
             'already','year','like','send','yes','last','right','also'
             ,'give','know','would','get','make','wonderful','phone',
             'want','something','heard','see','would','may','else',
             'one','day','minute','see word','today','inform','using',
             'call','good','morninig','anything','need','chat','thank','calling','please','moment',
             'check','welcome','nan','assigned','session','can','help','still','session','assist','inform',
             'number','checking','chatting','access','time','change','acknowledg',"customer","hello","just","minute","send","use","try"]
stop_wd=TEXT.ENGLISH_STOP_WORDS.union(stop)

# Create a CountVectorizer for parsing/counting words

In [7]:
tf=TfidfVectorizer(stop_words=stop_wd,ngram_range=(1,2),tokenizer=tokenize
                   ,max_df=0.7)                   
                 
mat_cv=tf.fit_transform(iter(document))


C:\Users\52100722\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['hear', 'wait', 'word'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [ ]:
#count_vectorizer = CountVectorizer(tokenizer=tokenize,stop_words=stop_wd,min_df=10,max_df=0.7,)
#mat_cv=count_vectorizer.fit_transform(iter(document))


In [8]:
#len(count_vectorizer.get_feature_names())
len(tf.get_feature_names())

64069

In [ ]:
#count_vectorizer.get_feature_names()

In [ ]:
#cv_vecs=pd.DataFrame(mat_cv.todense(), columns=count_vectorizer.get_feature_names()).head()
#cv_vecs=pd.DataFrame(mat_cv.todense(), columns=tf.get_feature_names()).head()


# Using LDA Model

In [9]:
# Convert sparse matrix of counts to a gensim corpus for LDA Model

cv_2corpus = matutils.Sparse2Corpus(mat_cv.transpose())
ld_words2idf=dict((v,k) for k,v in tf.vocabulary_.items())
ld_words2idf=corpora.Dictionary.from_corpus(corpus=cv_2corpus,id2word=ld_words2idf)



2019-04-29 12:15:34,324 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-04-29 12:15:34,381 : INFO : adding document #10000 to Dictionary(0 unique tokens: [])
2019-04-29 12:15:34,432 : INFO : adding document #20000 to Dictionary(0 unique tokens: [])
2019-04-29 12:15:34,480 : INFO : adding document #30000 to Dictionary(0 unique tokens: [])
2019-04-29 12:15:34,518 : INFO : adding document #40000 to Dictionary(0 unique tokens: [])
2019-04-29 12:15:34,564 : INFO : adding document #50000 to Dictionary(0 unique tokens: [])
2019-04-29 12:15:34,588 : INFO : adding document #60000 to Dictionary(0 unique tokens: [])
2019-04-29 12:15:34,636 : INFO : adding document #70000 to Dictionary(0 unique tokens: [])
2019-04-29 12:15:34,704 : INFO : adding document #80000 to Dictionary(0 unique tokens: [])
2019-04-29 12:15:34,751 : INFO : adding document #90000 to Dictionary(0 unique tokens: [])
2019-04-29 12:15:34,804 : INFO : adding document #100000 to Dictionary(0 unique tokens: [])
20

In [10]:
import multiprocessing

multiprocessing.cpu_count()

8

In [11]:
# Using LDA Model
logging.basicConfig(filename='~/gensim.log',
                    format="%(asctime)s:%(levelname)s:%(message)s",
                    level=logging.INFO)

lda=ldamulticore.LdaMulticore(cv_2corpus,id2word=ld_words2idf,num_topics=10,passes=50,eval_every=None,workers=7,
                              chunksize=10000)

2019-04-29 12:15:48,931 : INFO : using symmetric alpha at 0.1
2019-04-29 12:15:48,932 : INFO : using symmetric eta at 0.1
2019-04-29 12:15:48,940 : INFO : using serial LDA version on this node
2019-04-29 12:15:48,994 : INFO : running online LDA training, 10 topics, 50 passes over the supplied corpus of 107517 documents, updating every 70000 documents, evaluating every ~0 documents, iterating 50x with a convergence threshold of 0.001000
2019-04-29 12:15:48,996 : INFO : training LDA model using 7 processes
2019-04-29 12:15:49,281 : INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #10000/107517, outstanding queue size 1
2019-04-29 12:15:50,783 : INFO : PROGRESS: pass 0, dispatched chunk #1 = documents up to #20000/107517, outstanding queue size 2
2019-04-29 12:15:50,835 : INFO : PROGRESS: pass 0, dispatched chunk #2 = documents up to #30000/107517, outstanding queue size 3
2019-04-29 12:15:50,886 : INFO : PROGRESS: pass 0, dispatched chunk #3 = documents up to #40000/107517,

2019-04-29 12:16:18,922 : INFO : topic #7 (0.100): 0.022*"end" + 0.005*"think" + 0.004*"screen" + 0.004*"understand" + 0.003*"thx" + 0.003*"benefit" + 0.003*"nice" + 0.003*"option" + 0.003*"dental" + 0.003*"plan"
2019-04-29 12:16:18,926 : INFO : topic diff=0.148482, rho=0.280037
2019-04-29 12:16:18,979 : INFO : PROGRESS: pass 2, dispatched chunk #0 = documents up to #10000/107517, outstanding queue size 1
2019-04-29 12:16:19,676 : INFO : PROGRESS: pass 2, dispatched chunk #1 = documents up to #20000/107517, outstanding queue size 2
2019-04-29 12:16:20,410 : INFO : PROGRESS: pass 2, dispatched chunk #2 = documents up to #30000/107517, outstanding queue size 3
2019-04-29 12:16:21,197 : INFO : PROGRESS: pass 2, dispatched chunk #3 = documents up to #40000/107517, outstanding queue size 4
2019-04-29 12:16:22,018 : INFO : PROGRESS: pass 2, dispatched chunk #4 = documents up to #50000/107517, outstanding queue size 4
2019-04-29 12:16:22,874 : INFO : PROGRESS: pass 2, dispatched chunk #5 = do

2019-04-29 12:16:41,495 : INFO : topic diff=0.112245, rho=0.260363
2019-04-29 12:16:41,634 : INFO : PROGRESS: pass 4, dispatched chunk #0 = documents up to #10000/107517, outstanding queue size 1
2019-04-29 12:16:42,336 : INFO : PROGRESS: pass 4, dispatched chunk #1 = documents up to #20000/107517, outstanding queue size 2
2019-04-29 12:16:43,062 : INFO : PROGRESS: pass 4, dispatched chunk #2 = documents up to #30000/107517, outstanding queue size 3
2019-04-29 12:16:43,911 : INFO : PROGRESS: pass 4, dispatched chunk #3 = documents up to #40000/107517, outstanding queue size 4
2019-04-29 12:16:44,805 : INFO : PROGRESS: pass 4, dispatched chunk #4 = documents up to #50000/107517, outstanding queue size 4
2019-04-29 12:16:45,648 : INFO : PROGRESS: pass 4, dispatched chunk #5 = documents up to #60000/107517, outstanding queue size 4
2019-04-29 12:16:46,454 : INFO : PROGRESS: pass 4, dispatched chunk #6 = documents up to #70000/107517, outstanding queue size 4
2019-04-29 12:16:47,398 : INFO

2019-04-29 12:17:03,267 : INFO : PROGRESS: pass 6, dispatched chunk #0 = documents up to #10000/107517, outstanding queue size 1
2019-04-29 12:17:03,995 : INFO : PROGRESS: pass 6, dispatched chunk #1 = documents up to #20000/107517, outstanding queue size 2
2019-04-29 12:17:04,715 : INFO : PROGRESS: pass 6, dispatched chunk #2 = documents up to #30000/107517, outstanding queue size 3
2019-04-29 12:17:05,567 : INFO : PROGRESS: pass 6, dispatched chunk #3 = documents up to #40000/107517, outstanding queue size 4
2019-04-29 12:17:06,434 : INFO : PROGRESS: pass 6, dispatched chunk #4 = documents up to #50000/107517, outstanding queue size 4
2019-04-29 12:17:07,283 : INFO : PROGRESS: pass 6, dispatched chunk #5 = documents up to #60000/107517, outstanding queue size 4
2019-04-29 12:17:08,108 : INFO : PROGRESS: pass 6, dispatched chunk #6 = documents up to #70000/107517, outstanding queue size 4
2019-04-29 12:17:08,989 : INFO : PROGRESS: pass 6, dispatched chunk #7 = documents up to #80000/1

2019-04-29 12:17:25,453 : INFO : topic diff=0.085093, rho=0.230930
2019-04-29 12:17:25,499 : INFO : PROGRESS: pass 8, dispatched chunk #0 = documents up to #10000/107517, outstanding queue size 1
2019-04-29 12:17:26,212 : INFO : PROGRESS: pass 8, dispatched chunk #1 = documents up to #20000/107517, outstanding queue size 2
2019-04-29 12:17:26,934 : INFO : PROGRESS: pass 8, dispatched chunk #2 = documents up to #30000/107517, outstanding queue size 3
2019-04-29 12:17:27,688 : INFO : PROGRESS: pass 8, dispatched chunk #3 = documents up to #40000/107517, outstanding queue size 4
2019-04-29 12:17:28,543 : INFO : PROGRESS: pass 8, dispatched chunk #4 = documents up to #50000/107517, outstanding queue size 4
2019-04-29 12:17:29,458 : INFO : PROGRESS: pass 8, dispatched chunk #5 = documents up to #60000/107517, outstanding queue size 4
2019-04-29 12:17:30,404 : INFO : PROGRESS: pass 8, dispatched chunk #6 = documents up to #70000/107517, outstanding queue size 4
2019-04-29 12:17:31,381 : INFO

...

2019-04-29 12:23:04,338 : INFO : topic diff=0.035288, rho=0.140370
2019-04-29 12:23:04,392 : INFO : PROGRESS: pass 40, dispatched chunk #0 = documents up to #10000/107517, outstanding queue size 1
2019-04-29 12:23:05,094 : INFO : PROGRESS: pass 40, dispatched chunk #1 = documents up to #20000/107517, outstanding queue size 2
2019-04-29 12:23:05,780 : INFO : PROGRESS: pass 40, dispatched chunk #2 = documents up to #30000/107517, outstanding queue size 3
2019-04-29 12:23:06,512 : INFO : PROGRESS: pass 40, dispatched chunk #3 = documents up to #40000/107517, outstanding queue size 4
2019-04-29 12:23:07,263 : INFO : PROGRESS: pass 40, dispatched chunk #4 = documents up to #50000/107517, outstanding queue size 4
2019-04-29 12:23:08,031 : INFO : PROGRESS: pass 40, dispatched chunk #5 = documents up to #60000/107517, outstanding queue size 4
2019-04-29 12:23:08,793 : INFO : PROGRESS: pass 40, dispatched chunk #6 = documents up to #70000/107517, outstanding queue size 4
2019-04-29 12:23:09,631

2019-04-29 12:23:24,199 : INFO : topic diff=0.034433, rho=0.137683
2019-04-29 12:23:24,322 : INFO : PROGRESS: pass 42, dispatched chunk #0 = documents up to #10000/107517, outstanding queue size 1
2019-04-29 12:23:24,984 : INFO : PROGRESS: pass 42, dispatched chunk #1 = documents up to #20000/107517, outstanding queue size 2
2019-04-29 12:23:25,655 : INFO : PROGRESS: pass 42, dispatched chunk #2 = documents up to #30000/107517, outstanding queue size 3
2019-04-29 12:23:26,364 : INFO : PROGRESS: pass 42, dispatched chunk #3 = documents up to #40000/107517, outstanding queue size 4
2019-04-29 12:23:27,095 : INFO : PROGRESS: pass 42, dispatched chunk #4 = documents up to #50000/107517, outstanding queue size 4
2019-04-29 12:23:27,846 : INFO : PROGRESS: pass 42, dispatched chunk #5 = documents up to #60000/107517, outstanding queue size 4
2019-04-29 12:23:28,687 : INFO : PROGRESS: pass 42, dispatched chunk #6 = documents up to #70000/107517, outstanding queue size 4
2019-04-29 12:23:29,553

2019-04-29 12:23:44,216 : INFO : topic diff=0.033636, rho=0.135145
2019-04-29 12:23:44,269 : INFO : PROGRESS: pass 44, dispatched chunk #0 = documents up to #10000/107517, outstanding queue size 1
2019-04-29 12:23:44,963 : INFO : PROGRESS: pass 44, dispatched chunk #1 = documents up to #20000/107517, outstanding queue size 2
2019-04-29 12:23:45,646 : INFO : PROGRESS: pass 44, dispatched chunk #2 = documents up to #30000/107517, outstanding queue size 3
2019-04-29 12:23:46,423 : INFO : PROGRESS: pass 44, dispatched chunk #3 = documents up to #40000/107517, outstanding queue size 4
2019-04-29 12:23:47,172 : INFO : PROGRESS: pass 44, dispatched chunk #4 = documents up to #50000/107517, outstanding queue size 4
2019-04-29 12:23:47,993 : INFO : PROGRESS: pass 44, dispatched chunk #5 = documents up to #60000/107517, outstanding queue size 4
2019-04-29 12:23:48,714 : INFO : PROGRESS: pass 44, dispatched chunk #6 = documents up to #70000/107517, outstanding queue size 4
2019-04-29 12:23:49,578

2019-04-29 12:24:05,268 : INFO : PROGRESS: pass 46, dispatched chunk #1 = documents up to #20000/107517, outstanding queue size 2
2019-04-29 12:24:05,957 : INFO : PROGRESS: pass 46, dispatched chunk #2 = documents up to #30000/107517, outstanding queue size 3
2019-04-29 12:24:06,674 : INFO : PROGRESS: pass 46, dispatched chunk #3 = documents up to #40000/107517, outstanding queue size 4
2019-04-29 12:24:07,406 : INFO : PROGRESS: pass 46, dispatched chunk #4 = documents up to #50000/107517, outstanding queue size 4
2019-04-29 12:24:08,211 : INFO : PROGRESS: pass 46, dispatched chunk #5 = documents up to #60000/107517, outstanding queue size 4
2019-04-29 12:24:09,039 : INFO : PROGRESS: pass 46, dispatched chunk #6 = documents up to #70000/107517, outstanding queue size 4
2019-04-29 12:24:09,929 : INFO : PROGRESS: pass 46, dispatched chunk #7 = documents up to #80000/107517, outstanding queue size 3
2019-04-29 12:24:10,722 : INFO : PROGRESS: pass 46, dispatched chunk #8 = documents up to 

2019-04-29 12:24:25,234 : INFO : PROGRESS: pass 48, dispatched chunk #1 = documents up to #20000/107517, outstanding queue size 2
2019-04-29 12:24:25,917 : INFO : PROGRESS: pass 48, dispatched chunk #2 = documents up to #30000/107517, outstanding queue size 3
2019-04-29 12:24:26,619 : INFO : PROGRESS: pass 48, dispatched chunk #3 = documents up to #40000/107517, outstanding queue size 4
2019-04-29 12:24:27,365 : INFO : PROGRESS: pass 48, dispatched chunk #4 = documents up to #50000/107517, outstanding queue size 4
2019-04-29 12:24:28,211 : INFO : PROGRESS: pass 48, dispatched chunk #5 = documents up to #60000/107517, outstanding queue size 4
2019-04-29 12:24:29,016 : INFO : PROGRESS: pass 48, dispatched chunk #6 = documents up to #70000/107517, outstanding queue size 4
2019-04-29 12:24:29,799 : INFO : PROGRESS: pass 48, dispatched chunk #7 = documents up to #80000/107517, outstanding queue size 4
2019-04-29 12:24:30,674 : INFO : PROGRESS: pass 48, dispatched chunk #8 = documents up to 

In [12]:
lda.print_topics(num_words=15,num_topics=10)

2019-04-29 12:53:13,080 : INFO : topic #0 (0.100): 0.431*"leave" + 0.006*"nope" + 0.002*"term" + 0.002*"disability" + 0.002*"term disability" + 0.002*"online" + 0.001*"blue" + 0.001*"short" + 0.001*"log" + 0.001*"pin" + 0.001*"long term" + 0.001*"john" + 0.001*"cross" + 0.001*"blue cross" + 0.001*"short term"
2019-04-29 12:53:13,083 : INFO : topic #1 (0.100): 0.020*"insurance" + 0.012*"health" + 0.010*"coverage" + 0.009*"health insurance" + 0.008*"card" + 0.006*"don" + 0.005*"email" + 0.005*"cover" + 0.005*"plan" + 0.004*"tab" + 0.004*"push" + 0.004*"email push" + 0.004*"medical" + 0.004*"insurance card" + 0.004*"life"
2019-04-29 12:53:13,085 : INFO : topic #2 (0.100): 0.013*"account" + 0.013*"correct" + 0.011*"payment" + 0.006*"benefit" + 0.006*"question" + 0.005*"pull" + 0.005*"receive" + 0.004*"permission" + 0.004*"online" + 0.004*"look" + 0.004*"pull account" + 0.004*"permission browse" + 0.004*"days" + 0.004*"browse" + 0.004*"mail"
2019-04-29 12:53:13,087 : INFO : topic #3 (0.100)

[(0,
  '0.431*"leave" + 0.006*"nope" + 0.002*"term" + 0.002*"disability" + 0.002*"term disability" + 0.002*"online" + 0.001*"blue" + 0.001*"short" + 0.001*"log" + 0.001*"pin" + 0.001*"long term" + 0.001*"john" + 0.001*"cross" + 0.001*"blue cross" + 0.001*"short term"'),
 (1,
  '0.020*"insurance" + 0.012*"health" + 0.010*"coverage" + 0.009*"health insurance" + 0.008*"card" + 0.006*"don" + 0.005*"email" + 0.005*"cover" + 0.005*"plan" + 0.004*"tab" + 0.004*"push" + 0.004*"email push" + 0.004*"medical" + 0.004*"insurance card" + 0.004*"life"'),
 (2,
  '0.013*"account" + 0.013*"correct" + 0.011*"payment" + 0.006*"benefit" + 0.006*"question" + 0.005*"pull" + 0.005*"receive" + 0.004*"permission" + 0.004*"online" + 0.004*"look" + 0.004*"pull account" + 0.004*"permission browse" + 0.004*"days" + 0.004*"browse" + 0.004*"mail"'),
 (3,
  '0.039*"cancel" + 0.016*"option" + 0.009*"event" + 0.009*"life" + 0.009*"reach" + 0.008*"reach option" + 0.008*"continue" + 0.008*"life event" + 0.007*"respond" +

In [15]:
# Transform the docs from the word space to the topic space (like "transform" in sklearn)
lda_corpus = lda[cv_2corpus]
pd.DataFrame(lda_corpus[1])


,0,1
0,0,0.1
1,1,0.1
2,2,0.1
3,3,0.1
4,4,0.1
5,5,0.1
6,6,0.1
7,7,0.1
8,8,0.1
9,9,0.1


In [17]:
# Store the documents' topic vectors in a list so we can take a peak
lda_docs = [doc for doc in lda_corpus]

In [18]:
print("Topic Values", lda_docs[0]) # print Topic Values for document 0 

Topic Values [(0, 0.1), (1, 0.1), (2, 0.1), (3, 0.1), (4, 0.1), (5, 0.1), (6, 0.1), (7, 0.1), (8, 0.1), (9, 0.1)]


In [19]:
lda.num_topics

10

In [20]:

#consolidated data frame with document number , topic_number, and highest topic score and related topic terms
number_of_topics=10
topics=lda.show_topics(num_topics=number_of_topics)
topic_df=pd.DataFrame.from_records(topics,columns=["topic_id","topic_terms"])

highest_score_topic_lda_docs=[sorted(i,key=lambda x: x[1],reverse=True)[0] for i in lda_docs]

topics_by_documnent_numbers=pd.DataFrame(data=highest_score_topic_lda_docs,columns=["topic_number","topic_score"])

topics_by_documnent_numbers=topics_by_documnent_numbers.join(topic_df,on='topic_number')
topics_by_documnent_numbers.drop("topic_id",axis=1,inplace=True)

docnument_number=topics_by_documnent_numbers.reset_index()

topics_by_documnent_numbers=topics_by_documnent_numbers.reset_index()

topics_by_documnent_numbers.rename(columns={"index":"document_number"},inplace=True)

In [21]:
topics_by_documnent_numbers.head(10)
topics_by_documnent_numbers.topic_number.value_counts()

0    58517
5     9710
2     5826
4     5789
1     5424
9     5239
3     5122
8     4724
6     4171
7     2995
Name: topic_number, dtype: int64

In [ ]:
#lda.get_document_topics()
#lda.get_document_topics(ld_words2idf)

In [17]:
import seaborn as sns
docs_per_topic=(topics_by_documnent_numbers)

In [23]:
#docs_per_topic.to_csv("docs_per_topic.csv")

In [26]:
#prints text and topic_terms
def get_document2topic(doc_number):
    
    return(topics_by_documnent_numbers["topic_terms"].iloc[doc_number],document[doc_number])

topic, text=get_document2topic(8)
print("\n"+"\033[1m")
print("Topic_terms:" ,(tokenize(topic)),"\n")
print("Document:",text,"...")



Topic_terms: ['information', 'benefit', 'center', 'unresponsive', 'think', 'service', 'benefit', 'service', 'service', 'center', 'wait', 'benefit', 'center'] 

Document: And am checking in on the case to see what the disposition is ...


In [25]:
# Consolidated dictionary of topic numbers to text

mapped_dictionary_document={}

for i in range(len(topics_by_documnent_numbers)):
    mapped_dictionary_document[i]=get_document2topic(i)

In [ ]:
#mapped_dictionary_document.values()

## Visualization

In [ ]:

import pyLDAvis.gensim

In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, cv_2corpus,ld_words2idf )

In [ ]:
top_topics = lda.top_topics(cv_2corpus, topn=20)